<a href="https://www.kaggle.com/code/busetmkaya/customer-segmentation-analysis?scriptVersionId=226682857" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Customer Data Exploration

### Objective
This notebook demonstrates essential SQL data exploration skills by analyzing a dataset containing customer information. The goal is to uncover patterns related to customer demographics and contact information.

In [ ]:
# Import libraries

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## Data loading and preview

# Load and preview the dataset
df = pd.read_csv("//kaggle/input/customerorders/MOCK_DATA.csv")
df.head()

In [ ]:
# Data cleaning

# Remove leading/trailing spaces
df['FirstName'] = df['FirstName'].str.strip()
df['Email'] = df['Email'].str.strip().str.lower()
df['Phone'] = df['Phone'].str.strip()

# Check for and drop duplicates
duplicates = df.duplicated(subset=['CustomerID', 'Email']).sum()
print(f"Duplicates found: {duplicates}")
df = df.drop_duplicates()

# Preview cleaned data
df.head()

## SQL Environment Setup

I'll use SQLite to explore this data using SQL queries.

In [ ]:
# Create an SQLite database in memory

conn = sqlite3.connect(':memory:')
df.to_sql('customers', conn, index=False)

In [ ]:
# Data exploration

query_total = '''
SELECT 
    COUNT(DISTINCT CustomerID) AS Total_Customers
FROM customers;
'''

total_customers = pd.read_sql(query_total, conn)
total_customers

In [ ]:
# Check for duplicates

query_duplicates = '''
SELECT 
    Email,
    COUNT(CustomerID) AS CustomerCount
FROM customers
GROUP BY Email
HAVING COUNT(CustomerID) > 1
ORDER BY CustomerCount DESC;
'''

In [ ]:
duplicate_emails = pd.read_sql(query_total, conn)
duplicate_emails.head()

In [ ]:
# Customer age analysis

query_age_distribution = '''
SELECT 
  CASE
    WHEN Age BETWEEN 18 AND 24 THEN '18-24'
    WHEN Age BETWEEN 25 AND 34 THEN '25-34'
    WHEN Age BETWEEN 35 AND 44 THEN '35-44'
    WHEN Age BETWEEN 45 AND 54 THEN '45-54'
    WHEN Age >= 55 THEN '55+'
    ELSE 'Unknown'
  END AS AgeGroup,
  COUNT(CustomerID) AS Count
FROM customers
GROUP BY AgeGroup
ORDER BY Count DESC;
'''

age_distribution = pd.read_sql(query_age_distribution, conn)
age_distribution

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=age_distribution, x='AgeGroup', y='Count', palette='viridis')

plt.xlabel("Age Group")
plt.ylabel("Number of Customers")
plt.title("Customer Age Distribution")

for i, val in enumerate(age_distribution['Count']):
    plt.text(i, val + max(age_distribution['Count'])*0.01, f"{val}", ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# Most mopular email providers

query_providers = '''
SELECT 
    SUBSTR(Email, INSTR(Email, '@')+1) AS Provider, 
    COUNT(*) as Count
FROM customers
GROUP BY Provider
ORDER BY COUNT(*) DESC
LIMIT 10;
'''

email_providers = pd.read_sql(query_providers, conn)
email_providers

In [ ]:
# Visualization

plt.figure(figsize=(10,6))
sns.barplot(data=email_providers, x='Count', y='Provider', palette='crest')

plt.xlabel("Number of Customers")
plt.ylabel("Email Providers")
plt.title("Top Email Providers Used by Customers")

plt.tight_layout()
plt.show()

## Conclusion

This customer data exploration provided demographic information about our customer base:

- **Age Distribution**:
  - The customer demographic is predominantly mature (55+), indicating that products or services may appeal to older adults.
  - Younger age groups represent smaller segments, indicating opportunities to expand marketing efforts to younger demographics if desired.

- **Email Providers**:
  - Customers predominantly use less-common, possibly international or specialized email domains (such as `over-blog.com`, `umn.edu`, `taobao.com`).
  - This might suggest a specialized audience or international customer base, indicating a niche market.

- **Location Data**:
  - The dataset includes overly specific addresses (street numbers and house addresses), limiting useful analysis at broader geographical levels.
  - To enhance future analysis, broader location information (city, state, or region) is recommended.

Further analysis could benefit from broader geographic categorization and demographic details (e.g., purchasing behavior) for deeper customer segmentation.

